# Modelo Bert de Perguntas e Respostas

# Prova de Conceito  - BI Master PUC-RIO
# Utilização de modelo Hugging Face Transformer em Português para implementar um sistema de Perguntas e Respostas utilizando um texto do Wikipedia como contexto

Aluno: Nelson Custódio (2019-1)

Orientador : PhD Leonardo Alfredo Forero Mendoza

Data: 24/02/2021

## 1) Instala as Bibliotecas

In [1]:
!pip install transformers
!pip install torch
!pip install wikipedia

     |████████████████████████████████| 1.9MB 5.6MB/s 
     |████████████████████████████████| 890kB 34.2MB/s 
     |████████████████████████████████| 3.2MB 34.1MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp37-none-any.whl size=893262 sha256=0b1edceb0f29228bdb9e007f3dae2af7b5607d90fca6d3e5b215c3a03075278b
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses
  Created wheel for wikipedia: filename=wikipedia-1.4.0-cp37-none-any.whl size=11686 sha256=57e4258104c877f14844ca09744b1335626965976971bae464ba863ae4082efe
  Stored in directory: /root/.cache/pip/wheels/87/2a/18/4e471fd96d12114d16fe4a446d00c3b38fb9efcb744bd31f4a
Successfully built wikipedia


In [2]:
from transformers import AutoTokenizer, AutoModelForQuestionAnswering
from collections import OrderedDict
import torch
import wikipedia as wiki
import pprint as pp

# 2) Classe HuggingFace que quebra o texto em trechos de 512 tokes e executa o codigo de identificação das respostas possiveis para as perguntas realizadas.

In [3]:
class DocumentReader:
    def __init__(self, modelo_ou_caminho):
        self.READER_PATH = modelo_ou_caminho # diretório onde está o modelo pré-treinado
        self.tokenizer = AutoTokenizer.from_pretrained(self.READER_PATH) # Tokenizar o modelo
        self.model = AutoModelForQuestionAnswering.from_pretrained(self.READER_PATH) # aplica modelo
        self.comprimento_max = self.model.config.max_position_embeddings # tamanho maximo do modelo (padrão = 512 bytes)
        self.chunked = False

    def tokenize(self, question, text):
        self.inputs = self.tokenizer.encode_plus(question, text, add_special_tokens=True, return_tensors="pt") # Realiza a tokenização das questões e do contexto
        self.input_ids = self.inputs["input_ids"].tolist()[0] # lista de tokens

        if len(self.input_ids) > self.comprimento_max: # testa se o texto é maior que o comprimento maximo 
            self.inputs = self.trechando()
            self.trecho = True


    def trechando(self): # Faz a quebra do texto em trechos com comprimento maximo de 512 palavras

    #Cada um dos trechos indexados alimenta o modelo de forma a obtermos uma "resposta" para cada bloco. 
    #Se tivermos respostas não uteis o modelo retornará o token [CLS]
  
        qmask = self.inputs['token_type_ids'].lt(1) # tensor com valores em True ou False
        qt = torch.masked_select(self.inputs['input_ids'], qmask) # seleciona os tokens vinculados ao label 'input_ids'
        trecho_size = self.comprimento_max - qt.size()[0] - 1 # tamanho do trecho 

        # cria um dicionario para cada trecho, cada sub-dicionario contem a estrutura do modelo de trechos
        trecho_input = OrderedDict() # ordena os trechos
        for k,v in self.inputs.items():
            q = torch.masked_select(v, qmask) 
            c = torch.masked_select(v, ~qmask)
            trechos = torch.split(c, trecho_size)
            
            for i, trecho in enumerate(trechos):
                if i not in trecho_input:
                    trecho_input[i] = {}

                thing = torch.cat((q, trecho))
                if i != len(trechos)-1:
                    if k == 'input_ids':
                        thing = torch.cat((thing, torch.tensor([102])))
                    else:
                        thing = torch.cat((thing, torch.tensor([1])))

                trecho_input[i][k] = torch.unsqueeze(thing, dim=0)
        return trecho_input

    def get_answer(self):

       # busca as respostas por trecho

        if self.trecho:
            answer = ''
            for k, trecho in self.inputs.items():
                answer_start_scores, answer_end_scores = self.model(**trecho)[0], self.model(**trecho)[1]

                answer_start = torch.argmax(answer_start_scores)
                answer_end = torch.argmax(answer_end_scores) + 1

                ans = self.convert_ids_to_string(trecho['input_ids'][0][answer_start:answer_end])
                if ans != '[CLS]':
                    answer += ans + " / "
            return answer
        else:
            answer_start_scores, answer_end_scores = self.model(**self.inputs)

            answer_start = torch.argmax(answer_start_scores)
            answer_end = torch.argmax(answer_end_scores) + 1 
        
            return self.convert_ids_to_string(self.inputs['input_ids'][0][
                                              answer_start:answer_end])
          
    def convert_ids_to_string(self, input_ids): # converte os tokens em texto novamente
        return self.tokenizer.convert_tokens_to_string(self.tokenizer.convert_ids_to_tokens(input_ids))

In [4]:
#Desliga o aviso de comprimento da sequencia de tokenização, tornando a saida mais limpa
import logging
logging.getLogger("transformers.tokenization_utils").setLevel(logging.ERROR)

# 3) Carrega do texto da Wikipedia

In [5]:
wiki.set_lang('pt') # Definimos a lingua do Wikipedia
assunto = 'Donald Trump' # Assunto a ser pesquisado
conteudo = wiki.search(assunto) # Busca na Wikipedia os textos relacionados ao assunto escolhido
#pp.pprint(conteudo)


#print("Resultados sobre o assunto na Wikipedia:\n" + assunto)

pagina = wiki.page(conteudo[0]) #Utilizaremos apenas a primeira referencia encontrada sobre o assunto pesquisado
text = pagina.summary # baixa apenas o sumario de todo o texto relacionado

print(f"\nThe {conteudo[0]} O Texto contém {len(text)} caracteres.") # imprime o tamanho do texto

pp.pprint(text) # imprime o texto baixado


The Donald Trump O Texto contém 6161 caracteres.
('Donald John Trump (Nova Iorque, 14 de junho de 1946) é um empresário, '
 'personalidade televisiva e político americano, que serviu como o 45.º '
 'presidente dos Estados Unidos. Na eleição de 2016, Trump foi eleito pelo '
 'Partido Republicano ao derrotar a candidata democrata Hillary Clinton no '
 'número de delegados do colégio eleitoral; no entanto, perdeu por mais de 2,8 '
 'milhões de votos, a maior derrota nas urnas de um presidente eleito na '
 'história do país. Ele foi empossado para o cargo em 20 de janeiro de 2017 e, '
 'aos 70 anos de idade, foi a pessoa mais velha a assumir a presidência até '
 'que, em 2021, Joe Biden, aos 78 anos, assumiu o cargo.Trump nasceu e cresceu '
 'no Queens, um dos cinco distritos da cidade de Nova Iorque, e recebeu um '
 'diploma de bacharel em economia da Wharton School da Universidade da '
 'Pensilvânia em 1968. Em 1971, recebeu de seu pai, Fred Trump, o controle da '
 'empresa de imóveis e

# 4) Perguntas escolhidas

In [6]:
#defição das questões que queremos buscar as respostas no texto
questions = [
    "Quando nasceu Donald Trump?",
    "Quem é Donald John Trump?",
    "O que Donald Trump construiu durante sua carreira?",
    "Que tipo de investigação Robert Mueller conduziu?",
    "O que Trump fez quando perdeu a eleição presidencial em 2020?",
    "O que aconteceu em Janeiro de 2021?",
    "Como Donald Trump conduziu a politica externa americana?"
]

# 5) Execução do Modelo

In [7]:
#Modelo pre-treinado que será utilizado 
reader = DocumentReader("mrm8488/bert-base-portuguese-cased-finetuned-squad-v1-pt") 

for question in questions:
    reader.tokenize(question, text) # executa função que busca as respostas perguntas definidas sobre o texto escolhido
    pp.pprint(f"Pergunta: {question}") # imprime a questão
    pp.pprint(f"Respostas: {reader.get_answer()}") #imprime as respostas identificadas em cada trecho
    print()


'Pergunta: Quando nasceu Donald Trump?'
'Respostas: 14 de junho de 1946 / '

'Pergunta: Quem é Donald John Trump?'
'Respostas: um empresario, personalidade televisiva e politico americano / '

'Pergunta: O que Donald Trump construiu durante sua carreira?'
('Respostas: torres de escritorio, hoteis, cassinos, campos de golfe e outras '
 'instalacoes com sua marca em todo o mundo / ')

'Pergunta: Que tipo de investigação Robert Mueller conduziu?'
'Respostas: economia / investigacao oficial / obstrucao de justica / '

'Pergunta: O que Trump fez quando perdeu a eleição presidencial em 2020?'
('Respostas: mais de 2, 8 milhoes de votos / assinou uma ordem executiva que '
 'proibia a entrada de cidadaos oriundos de sete paises de maioria muculmana '
 'nos estados unidos, citando razoes de seguranca. em questoes internas, ele '
 'assinou um pacote de macicos cortes de impostos para individuos e empresas, '
 'e ainda rescindiu a penalidade do mandato de seguro saude individual do '
 'affordable